In [4]:
import numpy as np
import matplotlib.pyplot as plt
import math 

## TP7 :

In [33]:
def simulation_w1_w2(T , N , rho):
    incr_1 = np.sqrt(T/N)*np.random.randn(N)
    W_1 = np.concatenate(([0.] ,np.cumsum(incr_1)))
    
    incr_2 = np.sqrt(T/N)*np.random.randn(N)
    W_3 = np.concatenate(([0.] ,np.cumsum(incr_2)))
    
    W_2 = rho*W_1 + np.sqrt(1- rho**2)*W_3 
    
    return (W_1 , W_2)


def schema_s_v(s_0 , v_0 ,  k_v , theta , T , N  , sigma , r , W_1 , W_2):
    
    # W_1 , W_2 = simulation_w1_w2(T,N,rho)
    
    S = np.zeros(N+1)
    V = np.zeros(N+1)
    S[0]= s_0 
    V[0] = v_0
    
    for k in range(1,N+1) :
        V[k] = V[k-1] + k_v*(theta - V[k-1])*(T/N) + sigma*np.sqrt(max(V[k-1] , 0))*(W_2[k] - W_2[k-1])
        S[k] = S[k-1] + r*S[k-1]*(T/N) + np.sqrt(max(V[k-1] , 0))*S[k-1]*(W_1[k] - W_1[k-1])
        
        
    return (S,V)



def calcul_W1_W2_suiv(W_1 , W_2 , taille1 , taille2) :
    """
    """

    


def approximate_A(t , S , N) :
    res = (t/N)*(sum(S) - 0.5*(S[0] + S[-1]))
    return res 
    
    
def payoff(T , S , K , N):
    
    return max((1/T)*approximate_A(T,S,N) - K , 0 )
    
    

def price_mc(M , s_0 , v_0 ,  k_v , theta , T , N , rho , sigma , r , K ):
    
    price = 0
    var_price = 0
    for _ in range(M) :
        W_1 , W_2 = simulation_w1_w2(T,N,rho)
        S,_ = schema_s_v(s_0 , v_0 ,  k_v , theta , T , N  , sigma , r , W_1 , W_2 )
        p = payoff(T , S , K ,N)
        price += p
        var_price += p**2
        
    price = price*np.exp(-r*T) / M 
    var_price = (np.exp(-2*r*T)/M)*var_price - price**2
    
    return price , var_price 


def MSE(M , s_0 , v_0 ,  k_v , theta , T , N , rho , sigma , r , K , price_exacte , M_2 ) :
    """
    
    """
    MSE = 0
    for _ in range(M_2) :
        S_M = price_mc(M , s_0 , v_0 ,  k_v , theta , T , N , rho , sigma , r , K)[0]
        MSE += (S_M - price_exacte)**2 

    MSE = MSE / M_2
    
    return MSE

    
def calcul_Ml(m , L , l , epsilon) :
    a = L/(m**l)
    b = epsilon**2
    return int(L/(a*b))

def price_mc_multi(s_0 , v_0 ,  k_v , theta , T ,  rho , sigma , r , K , m , epsilon ):
    """ 
    
    """
    L = int(log(1/epsilon) / log(m) )
    M_0 = calcul_Ml(m , L , 0 , epsilon)
    price = price_mc(M_0 ,s_0 , v_0 ,  k_v , theta , T , m**0 , rho , sigma , r , K )

    for l in range(1,L):
        Ml = calcul_Ml(m , L , l , epsilon)
        price_l = 0 
        for _ in range(1 , Ml):
            # les browniens de schéma à m**l dates : 
            W_1 , W_2 = simulation_w1_w2(T,m**l,rho)
            # schéma à m**l dates : 
            S,_ = schema_s_v(s_0 , v_0 ,  k_v , theta , T , m**l  , sigma , r , W_1 , W_2)
            price_l += payoff(T , S , K ,m**l)
            # schéma à m**(l-1) dates :
            S,_ = schema_s_v(s_0 , v_0 ,  k_v , theta , T , m**(l-1)  , sigma , r , W_1 , W_2)
            price_l -= payoff(T , S , K ,m**l)
        price_l /= Ml 
        price += price_l
    
    return price


def MSE_multi(s_0 , v_0 ,  k_v , theta , T ,  rho , sigma , r , K , L  ,m , epsilon):
    """ 
    
    """
    


In [21]:
r = 0.03
s_0  = 100
K = 110
T = 2
rho = -0.2
v_0 = 0.04
k_v = 2
theta = 0.04
sigma = 0.01
N = 100
M = 10000
W_1 , W_2 = simulation_w1_w2(T,N,rho)
price = price_mc(M ,s_0 , v_0 , k_v , theta , T , N , rho , sigma ,r, K )

print(price)

(3.822726569626148, 69.41374030605103)


In [36]:
# MSE : 0.009 ; 0.015 
prix_exacte = 3.847906
M_2 = 10
mse = MSE(M ,s_0 , v_0 , k_v , theta , T , N , rho , sigma ,r, K, prix_exacte , M_2 )
print(mse)


0.015974694019937535


In [37]:
# MSE : 
# mse = MSE(M ,s_0 , v_0 , k_v , theta , T , N , rho , sigma ,r, K )
# print(mse)



# MC Multi-niveaux : 
m = 4
epsilon = 10e-3

price_multi = price_mc_multi(s_0 , v_0 , k_v , theta , T , rho , sigma , r  , K , ,m , epsilon)
print(f"price_multi = {price_multi}")


# print(calcul_Ml(m , L , L , epsilon))

price_multi = [18.65294012 45.97107568]


In [ ]:
# MSE en fonction de temps de calcul : 
